In [1]:
import numpy as np
import pandas as pd

In [5]:
df_rl = pd.read_csv('data/final_test_data.csv')
categories = ['Underwear', 'Garment Upper body', 'Swimwear',
              'Garment Lower body', 'Nightwear', 'Shoes', 'Accessories',
              'Garment Full body', 'Socks & Tights', 'Unknown',
              'Underwear/nightwear']

df_rl.head()

,index,t_dat,customer_id,article_id,price,sales_channel_id,Purchase History,category_sequence,category,purchase_cat_sequence
0,0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,[794321007 852643003 852643001 870304002 75042...,['Garment Upper body' 'Garment Upper body' 'Ga...,Underwear,['Garment Upper body' 'Garment Upper body' 'Ga...
1,1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,[794321007 852643003 852643001 870304002 75042...,['Garment Upper body' 'Garment Upper body' 'Ga...,Underwear,['Garment Upper body' 'Garment Upper body' 'Ga...
2,165807,2018-09-24,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,[794321007 852643003 852643001 870304002 75042...,['Garment Upper body' 'Garment Upper body' 'Ga...,Underwear,['Garment Upper body' 'Garment Upper body' 'Ga...
3,6827145,2019-03-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,578020002,0.013542,2,[794321007 852643003 852643001 870304002 75042...,['Garment Upper body' 'Garment Upper body' 'Ga...,Garment Upper body,['Garment Upper body' 'Garment Upper body' 'Ga...
4,21953950,2020-02-03,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,723529001,0.025407,2,[794321007 852643003 852643001 870304002 75042...,['Garment Upper body' 'Garment Upper body' 'Ga...,Swimwear,['Garment Upper body' 'Garment Upper body' 'Ga...


In [9]:
# df_rl_lim = df_rl.head(100)
# df_rl_lim.to_csv('data/smol.csv')

In [4]:
def parse_sequence_string(s):
    n = len(s)
    s = s[1:n-1]
    final_list = []
    f = False
    temp = ""
    for i in s:
        if (i == "'"):
            if not f:
                temp = ""
                f = True
            else:
                final_list.append(temp)
                f = False
        elif i == ' ':
            if f:
                temp+=i
        else:
            temp+=i
    return final_list

                

In [6]:
class ThompsonSampling:
    def __init__(self, categories):
        self.num_arms = len(categories)
        self.categories = categories
        self.category_map = { categories[i]:i for i in range(len(categories)) }
        self.num_pulls = np.zeros(self.num_arms)
        self.num_successes = np.zeros(self.num_arms)
        
    def clear_variables(self):
        self.num_pulls = np.zeros(self.num_arms)
        self.num_successes = np.zeros(self.num_arms)
        
    def simulate(self, users_choices):
        num_rounds = len(users_choices)
        for i in range(num_rounds):
            sampled_probabilities = np.random.beta(self.num_successes + 1, self.num_pulls - self.num_successes + 1)
            chosen_arm = np.argmax(sampled_probabilities)

            actual_arm = self.category_map[users_choices[i]]
            reward = 0 # could we work on getting 
            if (actual_arm == chosen_arm):
                reward+=1
            
            self.num_pulls[chosen_arm] += 1
            self.num_successes[chosen_arm] += reward

        sampled_probabilities = np.random.beta(self.num_successes + 1, self.num_pulls - self.num_successes + 1)
        choose_next_arm = np.argmax(sampled_probabilities)

        return self.categories[choose_next_arm]
    
    def get_probabilities(self):
        return np.random.beta(self.num_successes + 1, self.num_pulls - self.num_successes + 1)

In [ ]:
# make unique

In [6]:
def category_recommendations(user_id, user_history=[], session_count = 0):
    

    ind = df_rl[df_rl['customer_id'] == user_id].iloc[0]
    users_choices_string = ind['category_sequence']

    users_choices = parse_sequence_string(users_choices_string)
    
    
    # All users
    TS_all = ThompsonSampling(categories)
    for i in df_rl['category_sequence']:
        users_choices = parse_sequence_string(i)
        TS_all.simulate(users_choices)
    all_user_probabilities = TS_all.get_probabilities()
    sorted_categories_all = np.array(categories)[np.argsort(all_user_probabilities)][::-1]

    
    # One user
    TS_user = ThompsonSampling(categories)
    users_choices = parse_sequence_string(df_rl['category_sequence'][0])
    TS_user.simulate(users_choices)
    user_probabilities = TS_user.get_probabilities()
    sorted_categories_user = np.array(categories)[np.argsort(user_probabilities)][::-1]

    print("user categories: ", sorted_categories_user)
    
    if len(user_history) > 0:
        TS_user.simulate(user_history)
        TS_all.simulate(user_history)

    # ensemble the probabilities
    factor = np.exp(-session_count)
    probs = factor * TS_all.get_probabilities() + (1- factor) * TS_user.get_probabilities()
    sorted_categories_ensemble = np.array(categories)[np.argsort(probs)][::-1]
  
    return sorted_categories_ensemble

In [27]:
category_recommendations('000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318', [], 0)

user categories:  ['Garment Full body' 'Swimwear' 'Cosmetic' 'Underwear/nightwear'
 'Interior textile' 'Garment Lower body' 'Underwear' 'Fun' 'Items'
 'Accessories' 'Stationery' 'Nightwear' 'Furniture' 'Unknown'
 'Garment Upper body' 'Shoes' 'Socks & Tights' 'Garment and Shoe care'
 'Bags']


array(['Garment Upper body', 'Underwear', 'Garment Lower body', 'Shoes',
       'Bags', 'Unknown', 'Garment Full body', 'Nightwear', 'Cosmetic',
       'Socks & Tights', 'Furniture', 'Underwear/nightwear', 'Swimwear',
       'Interior textile', 'Items', 'Garment and Shoe care', 'Stationery',
       'Accessories', 'Fun'], dtype=object)

In [28]:
test_list = np.random.choice(1000, ['Swimwear', 'Cosmetic', 'Items'])
category_recommendations('000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318', test_list*10, 5)


user categories:  ['Underwear' 'Accessories' 'Bags' 'Fun' 'Socks & Tights' 'Nightwear'
 'Garment Upper body' 'Shoes' 'Interior textile' 'Unknown'
 'Garment and Shoe care' 'Items' 'Cosmetic' 'Stationery' 'Furniture'
 'Garment Lower body' 'Underwear/nightwear' 'Garment Full body' 'Swimwear']


array(['Cosmetic', 'Fun', 'Accessories', 'Garment Full body', 'Underwear',
       'Garment Lower body', 'Stationery', 'Swimwear', 'Bags', 'Items',
       'Socks & Tights', 'Garment Upper body', 'Underwear/nightwear',
       'Interior textile', 'Garment and Shoe care', 'Nightwear',
       'Furniture', 'Shoes', 'Unknown'], dtype=object)